# Transformerの詳細

- この章では、Transformerの詳細な実装について記載されている。
- 実際にPyTorchを使って実装を進めていく

In [ ]:
!git clone https://github.com/nlp-with-transformers/notebooks.git
%cd notebooks
from install import *
install_requirements()

## 3.1 Transformerのアーキテクチャ

- オリジナルのTransformerはエンコーダ・デコーダアーキテクチャとなっている。
- もともと機械翻訳向けにこの構造をしている。

<img src="https://github.com/nokomoro3/book-ml-transformers/blob/b2c6b3672a0de61117eba41f17e95002f317077f/img/ml-transformers-chap03-transformer-anatomy_2022-08-25-22-02-10.png?raw=1" />

- エンコーダの特徴
  - 入力トークン系列を埋め込みベクトルの系列に変換する。
  - 埋め込みベクトルは、隠れ状態やコンテキストとも呼ばれる。
- デコーダの特徴
  - 埋め込みベクトルの系列を入力し、出力トークン系列を生成する。
  - デコーダの終了は、特別なEOSトークンに到達するまで継続される。

- その後エンコーダ・デコーダのそれぞれが独立したモデルとして適応されていくこととなる。

- エンコーダのみモデル
  - テキスト分類や固有表現認識といったタスクに使用される。
  - このアーキテクチャでは、与えられたあるトークンの結果が、前後双方のコンテキストに依存する。
  - これは双方向アテンションと呼ばれ、BERT系が該当する。
- デコーダのみモデル
  - 次の単語を文脈から予測するようなタスクに使用される。
  - このアーキテクチャでは、与えられたあるトークンの結果が、前方のコンテキストのみに依存する。
  - これは因果的もしくは自己回帰型アテンションと呼ばれ、GPT系が該当する。
- エンコーダ・デコーダモデル
  - 機械翻訳や要約といったタスクに使用される。
  - BARTやT5がこのアーキテクチャに該当する。

- 実際にはこれらの区別はあいまいであるので注意が必要。

## 3.2 エンコーダ

- Transformerのエンコーダは複数のエンコーダをスタックする。
- エンコーダは文脈情報を埋め込んだ表現を生成する。
- エンコーダは、マルチヘッドセルフアテンションと単純な順伝搬層で構成される。

<img src="https://github.com/nokomoro3/book-ml-transformers/blob/b2c6b3672a0de61117eba41f17e95002f317077f/img/ml-transformers-chap03-transformer-anatomy_2022-08-25-22-14-41.png?raw=1" />

### 3.2.1 セルフアテンション

- アテンションは系列の各要素に異なる重みを割り当てる機構である。
- テキストの場合、系列の要素はトークン埋め込みである。
- トークン埋め込みは、固定次元のベクトルで表される。
  - 例えばBERTの場合、768次元のベクトルとなる。
- セルフアテンションは、入力されるトークン埋め込み$x_j$すべてを使用した線形和を、その系列ぶん計算する。

$$ x^{'}_i = \sum^n_{j=1}{w_{ji}x_j}$$

- $w_{ji}$はアテンションの重みと呼ばれ、$\sum_{j}{w_{ji}}=1$となるように正規化される。
- セルフアテンションは以下のように、同一単語であっても周囲の文脈を考慮した埋め込みを生成することができる。

<img src="https://github.com/nokomoro3/book-ml-transformers/blob/b2c6b3672a0de61117eba41f17e95002f317077f/img/ml-transformers-chap03-transformer-anatomy_2022-08-26-09-09-37.png?raw=1" />

- この出力される結果を文脈埋め込みと呼び、Transformerに選考してELMoなどの言語モデルで取り入れられている。
  - [Deep Contextualized Word Representations (2018-02-15)](https://arxiv.org/abs/1802.05365)
- 以降は$w_{ji}$を計算する方法について述べる。

#### 3.2.1.1 スケール化ドット積アテンション

- いくつか実装があるがTransformerでは以下のステップを踏む。
  - 各トークン埋め込みをクエリ・キー・バリューというベクトルに射影する。
  - クエリとキーの類似度を計算し、これをアテンションスコアと呼ぶ。
    - アテンションスコアは、系列長をnとすると、n x n行列となる。
  - アテンションスコアは任意の数を生成できるため、softmaxで正規化して、これを重み$ w_{ji} $とする。
    - 正規化は入力系列方向に行い、$ \sum_{j}{w_{ji}}=1 $となるようにする
  - バリューベクトル$ v_1,...,v_n $の重み付け線形和$ x^{'}_i = \sum^n_{j=1}{w_{ji}v_j} $で文脈埋め込みを計算する
- これらの可視化をJupyter向けのBertVizにおけるneuron_viewで行うことができる。
  - layerはエンコーダスタックの番号、headは後述するマルチヘッドのいずれかを示すと思われる。


In [ ]:
from transformers import AutoTokenizer
from bertviz.transformers_neuron_view import BertModel
from bertviz.neuron_view import show

model_ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = BertModel.from_pretrained(model_ckpt)
text = "time flies like an arrow"
show(model, "bert", tokenizer, text, display_mode="light", layer=0, head=8)

- 実際にこれらをPyTorchで作成する。
- まずはtokenizerによりテキストをトークン化する。

In [ ]:
# 単純化のため、add_special_tokens=Falseにより[CLS]や[SEP]などのトークンを除外

inputs = tokenizer(text, return_tensors="pt", add_special_tokens=False)
inputs.input_ids

- nn.Embeddingにより、one-hot化と密ベクトル変換を同時に行う。

In [ ]:
from torch import nn
from transformers import AutoConfig

# BERTとパラメータを合わせるために、configファイルを読み込む
config = AutoConfig.from_pretrained(model_ckpt)

token_emb = nn.Embedding(config.vocab_size, config.hidden_size)
token_emb

- 未学習であるため初期値のままであるが、以下のように埋め込みベクトルを作成できる。

In [ ]:
inputs_embeds = token_emb(inputs.input_ids)
inputs_embeds.size()

- 埋め込みを射影し、類似度をドット積を使ってアテンションスコアを計算する。
  - 簡単のため、クエリ・キー・バリューが埋め込みベクトルと同じになっている。
  - 実際には後述するように、それぞれ独立した重み$ W_{Q,K,V} $を適用して生成する。

In [ ]:
import torch
from math import sqrt

query = key = value = inputs_embeds # 簡単のため同じものをquery, key, valueとしている。

# bmmはバッチ化された行列積を計算することができる
# sqrt(dim_k)でわるのはこれは必要なのかわからないが、
# 埋め込みのベクトルサイズで割ることで後段のsoftmaxが飽和しないようにしているらしい。
dim_k = key.size(-1)
scores = torch.bmm(query, key.transpose(1,2)) / sqrt(dim_k)
scores.size()

- softmaxを適用する。

In [ ]:
import torch.nn.functional as F

weights = F.softmax(scores, dim=-1)
weights.sum(dim=-1)

- weightsのvalueの行列積を計算する。

In [ ]:
attn_outputs = torch.bmm(weights, value)
attn_outputs.shape

- これらを関数化すると以下となる。

In [ ]:
def scaled_dot_product_attention(query, key, value):
    dim_k = query.size(-1)
    scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)
    weights = F.softmax(scores, dim=-1)
    return torch.bmm(weights, value)

#### 3.2.1.2 マルチヘッドアテンション

- この例ではquery, key, valueが同じベクトルであるため、その単語自身に非常に大きなスコアが割り当たる。
- 実際は文脈を考慮するために、各埋め込みに対して独立した3つの異なる線形射影をquery, key, valueに使用する。
- 各線形射影は学習可能なパラメータを持つ。
- これによりセルフアテンションは系列の意味・文脈を考慮できる。
- これらの線形射影は複数の集合を持つことで複数の関係性を考慮できるため有益である。
- 一つ一つをアテンションヘッドといいこれらをマルチヘッドアテンションと呼ぶ。
- 一つのヘッドのsoftmaxでは類似性の一面にしか着目できないため、複数に意味がある。
- あるヘッドが主語と動詞の相互作用、あるヘッドが近くの形容詞を見つけるなどが可能となると考えられている。
- マルチヘッドは最終的には図のように連結され全結合層で処理される。

![](https://github.com/nokomoro3/book-ml-transformers/blob/main/img/ml-transformers-chap03-transformer-anatomy_2022-08-27-08-43-44.png?raw=1)

- まずは一つのアテンションヘッドを定義する。

In [ ]:
class AttentionHead(nn.Module):
    def __init__(self, embed_dim, head_dim):
        super().__init__()
        self.q = nn.Linear(embed_dim, head_dim)
        self.k = nn.Linear(embed_dim, head_dim)
        self.v = nn.Linear(embed_dim, head_dim)

    def forward(self, hidden_state):
        attn_outputs = scaled_dot_product_attention(
            self.q(hidden_state), self.k(hidden_state), self.v(hidden_state))
        return attn_outputs

- head_dimは射影後の次元数で、トークン埋め込みの次元数より小さい必要はないが、
実際にはトークン埋め込み次元数をヘッドの数で割ったものを使用する。
- BERTの場合12個のヘッドがあるので、head_dim = 768/12 = 64となる。
- これを複数組み合わせて以下のように定義する。

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        embed_dim = config.hidden_size
        num_heads = config.num_attention_heads
        head_dim = embed_dim // num_heads

        self.heads = nn.ModuleList(
            [AttentionHead(embed_dim, head_dim) for _ in range(num_heads)]
        )

        self.output_linear = nn.Linear(embed_dim, embed_dim)

    def forward(self, hidden_state):
        x = torch.cat([h(hidden_state) for h in self.heads], dim=-1)
        x = self.output_linear(x)
        return x

- このクラスをテストします。
- configからパラメータを読み込めるように定義したため、BERTのconfigを渡すことで同じ設定を使用できます。

In [ ]:
multihead_attn = MultiHeadAttention(config)
attn_output = multihead_attn(inputs_embeds)
attn_output.size()

- 最後にアテンションスコアの可視化をBertVizのhead_viewで行う。

In [ ]:
from bertviz import head_view
from transformers import AutoModel

model = AutoModel.from_pretrained(model_ckpt, output_attentions=True)

sentence_a = "time flies like an arrow"
sentence_b = "fruit flies like a banana"

viz_inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt') # A,Bの文は[SEP]を挟んで連結される
attention = model(**viz_inputs).attentions
sentence_b_start = (viz_inputs.token_type_ids == 0).sum(dim=1) # UI用に文の境界を設定
tokens = tokenizer.convert_ids_to_tokens(viz_inputs.input_ids[0]) # IDをトークンに変換
head_view(attention, tokens, sentence_b_start, heads=[1,8]) # headsを指定すれば特定のヘッドのアテンションが確認できる

### 3.2.2 順伝搬層

- 順伝搬層は2つの全結合層で構成されます。
- 系列の位置情報は保持された状態で処理するため、一単位順伝搬層（position-wise feed-forward layer）と呼ばれる。
- CV分野の人からはカーネルサイズ1の1次元畳み込み層と呼ばれることもある。
- 実装としては、[batch_size, seq_len, hidden_dim]のままflattenせずに渡せば位置情報が保持される。

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.linear_1 = nn.Linear(config.hidden_size, config.intermediate_size)
        self.linear_2 = nn.Linear(config.intermediate_size, config.hidden_size)
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, x):
        x = self.linear_1(x)
        x = self.gelu(x)
        x = self.linear_2(x)
        x = self.dropout(x)
        return x

- 文献によれば、intermediate_sizeはhidden_dimの4倍で、活性化関数はGELUが使われる。
- intermediate_sizeはスケールアップする際に最も良くスケールされることが多い。
- また活性化関数は1層目の後にのみ使用され、2層目の後にはdropoutが使われる。

In [ ]:
feed_forward = FeedForward(config)
ff_outputs = feed_forward(attn_outputs)
ff_outputs.size()

### 3.2.3 レイヤー正規化の追加

- レイヤ正規化には2種類ある。

![](./img/ml-transformers-chap03-transformer-anatomy_2022-08-27-10-17-07.png)

- レイヤー後置型は不安定であるため、学習率を徐々に上げていく学習率のウォームアップの実施が必要。
- レイヤー前置型の方が安定かつ学習率のウォームアップが不要でありこちらを採用する。

In [ ]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.layer_norm_1 = nn.LayerNorm(config.hidden_size)
        self.layer_norm_2 = nn.LayerNorm(config.hidden_size)
        self.attention = MultiHeadAttention(config)
        self.feed_forward = FeedForward(config)

    def forward(self, x):
        # レイヤー正規化を適用し、入力をクエリ、キー、バリューにコピー
        hidden_state = self.layer_norm_1(x)
        # スキップ接続付きのアテンションを適用
        x = x + self.attention(hidden_state)
        # スキップ接続付きの順伝播層を適用
        x = x + self.feed_forward(self.layer_norm_2(x))
        return x

- スキップ接続も実施する。スキップ接続はレイヤー正規化前の値と加算している。
- これらを実際に処理すると以下となる。

In [ ]:
encoder_layer = TransformerEncoderLayer(config)
inputs_embeds.shape, encoder_layer(inputs_embeds).size()

- nn.LayerNormについて補足する。
- これはBatch Normalizationの改良版で、通常バッチ方向に平均0、分散1となるようなものが、Batch Normalizationである。
- 一方、Layer Normalizationは隠れ状態方向に平均0、分散1となるように正規化する。
- 以下で確認してみる。


In [ ]:
encoder_layer.layer_norm_1(inputs_embeds).mean(-1), encoder_layer.layer_norm_1(inputs_embeds).std(-1)

- 実際には、学習するパラメータなどやepsがあるため、正確な平均0、分散1にはなっていないがおおむね傾向を確認できる。
- Batch Normalizationを使わない理由は、バッチサイズが小さいときに不安定になる、系列処理の場合、学習時と推論時で系列長が異なる場合があることなどが挙げられる。

- ちなみに以下のGroup Normalizationの論文にあるように正確には
  - Batch Normalizationは、画像の場合ではバッチ方向とピクセル位置方向の正規化
  - Layer Normalizationは、画像の場合ではピクセル位置方向とチャンネル方向の正規化
  - Instance Normalizationは、画像の場合ではピクセル位置方向のみの正規化

- 以下がそれを示す図である。

![](./img/ml-transformers-chap03-transformer-anatomy_2022-08-27-14-44-43.png)

- この用語に沿う場合、Layer Normalizationといえるか微妙だが、nn.LayerNorm自体は任意の軸方向の正規化が可能であるため、Transformerの場合は隠れ状態方向に正規化することに使用されている。

- Group Normalizationの論文は以下である。
  - [https://arxiv.org/abs/1803.08494](https://arxiv.org/abs/1803.08494)